![Alt text](venice-logo-lion.png "Venice Logo")

# Welcome To Venice!

<div style="background-color:rgba(0, 0, 0, 0.0470588);">
This notebook is meant to serve as an interactive demo of Venice and some of it's concepts.  In this workbook we'll demo:

* Preparing and Processing a dataset with Spark
* Preparing a Venice store
* Pushing the dataset to Venice and querying it

To this end, we're going to demo a simple workflow that an AI engineer may go through when publishing processing and Publishing data to Venice.
First, we'll start by getting a dataset from Hugging Face.
    
</div>

## Preparing and Processing a dataset with Spark

<div style="background-color:rgba(0, 0, 0, 0.0470588);">
For this demo to be interesting we'll need an interesting dataset, and the means to prepare it.  We'll use spark and a dataset from [Hugging Face](https://huggingface.co/)

### What is Spark?

[Apache Spark](https://spark.apache.org/) is an open sourced data processing engine that is quite popular.  It enables users to set up data processing jobs in a distributed manner on a cluster (if you're familiar with Hadoop you probably understand some of the core concepts).  Venice supports writing data to a Venice store via Spark, and we will show case that capability here.  In the below cell, we'll use python and pyspark as our means to programmatically interact with a spark cluster that is running as part of this demo.

### What is our dataset?

We're going to grab a dataset from Hugging Face and upload it for processing to our spark cluster.  This dataset is a table of wine reviews!  Yay!
</div>

In [89]:
import requests
from pyspark import SparkFiles
from pyspark import SparkContext
from pyspark.sql import SparkSession

dataset = "alfredodeza/wine-ratings"

# Initialize our Spark Session
spark = SparkSession.builder.appName("VeniceWineReviewsDemo").getOrCreate()

# Query HuggingFace for the appropriate request paths for our dataset
HUGGING_FACE_PARQUET_API = "https://huggingface.co/api/datasets/{dataset}/parquet"
r = requests.get(HUGGING_FACE_PARQUET_API.format(dataset=dataset))
train_parquet_files = r.json()['default']['train']

# Download the datasets to our spark cluster
for url in train_parquet_files:
  spark.sparkContext.addFile(url)

# Process the parquet files on the spark cluster so we can query them
df = spark.read.parquet(SparkFiles.getRootDirectory() + "/*.parquet")

# Displaying first 10 rows of our dataset
df.show(n=10)

# It's normally good practice to stop your spark session at the end, however, we're
# going to reuse some of this state in the following cells, so we'll leave this commented
# out for now.  If for your own purposes you want to try and play with other datasets,
# be sure to run with spark.close between runs so as to clear out state and avoid errors.
# spark.stop()

+--------------------+--------------------+--------+------+--------------------+
|                name|              region| variety|rating|               notes|
+--------------------+--------------------+--------+------+--------------------+
|1000 Stories Bour...|Mendocino, Califo...|Red Wine|  91.0|This is a very sp...|
|1000 Stories Bour...|          California|Red Wine|  89.0|The California Go...|
|1000 Stories Bour...|          California|Red Wine|  90.0|The California Go...|
|1000 Stories Bour...|North Coast, Cali...|Red Wine|  91.0|The wine has a de...|
|1000 Stories Bour...|          California|Red Wine|  90.0|Batch #004 is the...|
|1000 Stories Bour...|          California|Red Wine|  91.0|1,000 Stories Bou...|
|1000 Stories Bour...|          California|Red Wine|  92.0|Batch 55 embodies...|
|12 Linajes Crianz...|Ribera del Duero,...|Red Wine|  92.0|Red with violet h...|
|12 Linajes Reserv...|Ribera del Duero,...|Red Wine|  94.0|On the nose, a co...|
|14 Hands Cabernet...|Columb

### Using Spark To Prepare Our Dataset

<div style="background-color:rgba(0, 0, 0, 0.0470588);">

Now that we have our dataset, we need to manipulate it a bit. 

Venice is a key/value data store which uses [Avro](https://avro.apache.org/) as it's serialization format.  To that end, we'll need to process and transform our dataset to have a key/value structure and serialize it to Avro.

The key to our dataset in Venice needs to be both non nullable and unique.  So we'll need to make sure those constraints are applied to our dataset before we can push it.  Looking at our dataset, we want to select a field for our key.  'name' seems a likely choice (and who wouldn't want a database where you can look up a wine rating by it's name!).  So we'll have spark process this data to form key/value pairs by grouping the columns in our dataset, remove null entries and duplicates, and finally serialize the dataset to avro.

</div>

In [90]:
from pyspark.sql.functions import struct, col
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# Create a struct type for the value column and group together the columns
# which will make up our field value in the Venice store.
df_transformed = df.withColumn("value", struct(
    col("region").alias("region"),
    col("variety").alias("variety"),
    col("rating").alias("rating"),
    col("notes").alias("notes")
)).select(
    col("name").alias("key"),
    col("value")
).dropDuplicates(["key"])

# Define the new schema with key as non-nullable
updated_schema = StructType([
    StructField("key", StringType(), nullable=False),
    StructField("value", StructType([
        StructField("region", StringType(), nullable=True),
        StructField("variety", StringType(), nullable=True),
        StructField("rating", FloatType(), nullable=True),
        StructField("notes", StringType(), nullable=True)
    ]), nullable=False)
])

# Create new DataFrame with updated schema
df_non_nullable_key = spark.createDataFrame(df_transformed.rdd, schema=updated_schema)

# Show the resulting DataFrame schema
df_non_nullable_key.printSchema()

df_non_nullable_key.show(n=3)
df_non_nullable_key.write.format("avro").save("transformed_avro")

root
 |-- key: string (nullable = false)
 |-- value: struct (nullable = false)
 |    |-- region: string (nullable = true)
 |    |-- variety: string (nullable = true)
 |    |-- rating: float (nullable = true)
 |    |-- notes: string (nullable = true)

+--------------------+--------------------+
|                 key|               value|
+--------------------+--------------------+
|1000 Stories Bour...|{California, Red ...|
|1000 Stories Bour...|{North Coast, Cal...|
|1000 Stories Bour...|{California, Red ...|
+--------------------+--------------------+
only showing top 3 rows



## Preparing a Venice store

<div style="background-color:rgba(0, 0, 0, 0.0470588);">
Looking good!  So we've got our data set.  Now we need to prepare a Venice store.  We've set up some endpoints in this demo that will enable you to interact with a venice cluster called "venice-cluster0".  We'll create a store called "wine-ratings-store" and we'll initialize the store with a schema that matches our dataset.  It's possible to generate the schema from the parqet file in the spark session, but to simplify this demo we've already generated the schema files.  We'll create our venice store with the below code:
</div>


In [84]:
import subprocess
from subprocess import Popen, PIPE, STDOUT

# Utility function for printing command output
def log_subprocess_output(pipe):
    for line in iter(pipe.readline, b''):
        print(line)

# Arguments for Venice store creation
create_store_args = [
    "./create-store.sh",
    "http://venice-controller:5555", 
    "venice-cluster0",
    "wine-ratings-store", # Name of our store
    "~/wineKeySchema.avsc",
    "~/wineValueSchema.avsc"
]

# Submit the job
process = Popen(create_store_args, stdout=PIPE, stderr=STDOUT)
with process.stdout:
    log_subprocess_output(process.stdout)
exitcode = process.wait()

b'[WARN] Running admin tool without SSL.\n'
b'WARNING: sun.reflect.Reflection.getCallerClass is not supported. This will impact performance.\n'
b'2024-10-31 21:34:27 INFO [ControllerClient] [main] Discovered leader controller http://venice-controller:5555 from http://venice-controller:5555\n'
b'{\n'
b'  "error" : "Store wine-ratings-store already exists"\n'
b'}\n'
b'Exception in thread "main" com.linkedin.venice.exceptions.VeniceException: Store wine-ratings-store already exists\n'
b'\tat com.linkedin.venice.AdminTool.verifyStoreExistence(AdminTool.java:2347)\n'
b'\tat com.linkedin.venice.AdminTool.createNewStore(AdminTool.java:828)\n'
b'\tat com.linkedin.venice.AdminTool.main(AdminTool.java:292)\n'
b'[WARN] Running admin tool without SSL.\n'
b'2024-10-31 21:34:28 INFO [ControllerClient] [main] Discovered leader controller http://venice-controller:5555 from http://venice-controller:5555\n'
b'{"status":"success"}\n'


## Pushing the dataset to Venice and querying it

<div style="background-color:rgba(0, 0, 0, 0.0470588);">
And now the moment we've been waiting for!  We've prepared our dataset and we've created our venice store.  Now let's transmit the data to Venice via spark!
    
With this demo we've provided the push job as a java jar which we'll submit to our spark cluster.  We've also prepared a properties file with the push job which contains some of the mandatory configurations needed for the push (things like the name of the store, the cluster's endpoint, and the path to the data that we'll need to transmit).  You can take a look at these configurations in batch-push-job.properties in the home directory of this demo.
</div>

In [91]:
# Path to the push job jar Java JAR file
jar_path = "bin/venice-push-job-all.jar"

# Arguments for spark-submit
spark_submit_args = [
    "spark-submit",
    "--class", "com.linkedin.venice.hadoop.VenicePushJob",  # Push job main class
    jar_path,
    "batch-push-job.properties"  # Configurations
]

# Submit the job
process = Popen(spark_submit_args, stdout=PIPE, stderr=STDOUT)
with process.stdout:
    log_subprocess_output(process.stdout)
exitcode = process.wait()

b'24/10/31 21:49:01 INFO VenicePushJob: Constructing VenicePushJob: {\n'
b'\tcontroller.request.retry.attempts: 5, \n'
b'\tdata.writer.compute.job.class: com.linkedin.venice.hadoop.spark.datawriter.jobs.DataWriterSparkJob, \n'
b'\tinput.path: /home/jovyan/transformed_avro, \n'
b'\tpoll.job.status.interval.ms: 1000, \n'
b'\tpush.job.status.upload.enable: false, \n'
b'\tspark.native.input.format.enabled: true, \n'
b'\tvenice.discover.urls: http://venice-controller:5555, \n'
b'\tvenice.store.name: wine-ratings-store, \n'
b'\tvenice.writer.close.timeout.ms: 500\n'
b'}\n'
b'24/10/31 21:49:01 INFO VenicePushJob: Using /tmp/hadoop-jovyan/venice-push-job as shared temp directory\n'
b"24/10/31 21:49:01 INFO VenicePushJob: Using /tmp/hadoop-jovyan/venice-push-job as this job's temp directory\n"
b'24/10/31 21:49:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable\n'
b'24/10/31 21:49:01 INFO VenicePushJob: Going to use con

### Query the data

<div style="background-color:rgba(0, 0, 0, 0.0470588);">
And now finall, let's query this dataset from Venice!!
    </div>

In [92]:
# Query the store for the data
query_store_args = [
    "./fetch.sh",
    "http://venice-router:7777", 
    "wine-ratings-store",
    "A. Mano Puglia Primitivo 2006",
]

# Submit the job
process = Popen(query_store_args, stdout=PIPE, stderr=STDOUT)
with process.stdout:
    log_subprocess_output(process.stdout)
exitcode = process.wait()

b'\n'
b'com/linkedin/avro/fastserde/generated/deserialization/AVRO_1_10/MultiGetResponseRecordV1_GenericDeserializer_433464781_433464781.java\n'
b'Key string parsed successfully. About to make the query.\n'
b'com/linkedin/avro/fastserde/generated/deserialization/AVRO_1_10/value_GenericDeserializer_1319288651_1319288651.java\n'
b'key-class=java.lang.String\n'
b'value-class=org.apache.avro.generic.GenericData.Record\n'
b'request-path=storage/wine-ratings-store/OkEuIE1hbm8gUHVnbGlhIFByaW1pdGl2byAyMDA2?f=b64\n'
b'key=A. Mano Puglia Primitivo 2006\n'
b'value={"region": "Italy", "variety": "Red Wine", "rating": 88.0, "notes": "Made from 100% Primitivo (Zinfandel\'s ancestor) from Puglia. A. Mano (meaning \\"made by hand\\") is made from 70-100 year-old, hand-cultivated, low-yielding vines. "}\n'


__Saluti!!__

In [93]:
# Don't forget to clean up your spark session afterward
spark.stop()